In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(3,16,3,padding=1),
            nn.ELU(),
            #nn.BatchNorm2d(16),
            #nn.Dropout2d(0.2),
            nn.Conv2d(16,32,3,padding=1),
            nn.ELU(),
            #nn.BatchNorm2d(32),
            #nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,3,padding=1),
            nn.ELU(),
            #nn.BatchNorm2d(64),
            #nn.Dropout2d(0.2),
            nn.MaxPool2d(2,2)
        )
        self.fc_layer = nn.Sequential(
            nn.Linear(64*8*8, 1024),
            nn.ELU(),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, 256),
            nn.ELU(),
            nn.BatchNorm1d(256),
            nn.Linear(256,64),
            nn.ELU(),
            nn.BatchNorm1d(64),
            nn.Linear(64,10)
        )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)
            if isinstance(m, nn.Linear):
                init.kaiming_normal(m.weight.data)
                m.bias.data.fill_(0)


    def forward(self, x):
        out = self.layer(x)
        out = out.view(out.shape[0], -1)
        out = self.fc_layer(out)

        return out

    

batch_size = 64
learning_rate = 0.002
num_epoch = 1000



cifar_train = dset.CIFAR10("CIFAR10/", train=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
]), target_transform=None, download=True)
cifar_test = dset.CIFAR10("CIFAR10/", train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,0.5,0.5), std=(0.5,0.5,0.5)),
]), target_transform=None, download=True)


def ComputeAccr(dloader, imodel):
    correct = 0
    total = 0

    for j, [imgs, labels] in enumerate(dloader):
        img = Variable(imgs, volatile=True).cuda()
        label = Variable(labels).cuda()

        output = imodel.forward(img)
        _, output_index = torch.max(output, 1)

        total += label.size(0)
        correct += (output_index == label).sum().float()

    print("## Accuracy of Test Data : {}".format(100*correct/total))


train_loader=torch.utils.data.DataLoader(list(cifar_train)[:], batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)
test_loader=torch.utils.data.DataLoader(cifar_test, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=True)

###################### TEST ######################

model = torch.load('.nets/REAL_1_L2E1000B64.pkl')
ComputeAccr(test_loader, model)
import sys
sys.exit(0)

###################### TEST ######################






###################### NOT EXECUTED ######################
###################### CODE FOR TRAIN  ######################
model = CNN().cuda()



loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.2)

model.train()
for i in range(num_epoch):
    for j, [image, label] in enumerate(train_loader):
        x = Variable(image).cuda()
        y_ = Variable(label).cuda()

        optimizer.zero_grad()
        output = model.forward(x)
        loss = loss_func(output, y_)
        loss.backward()
        optimizer.step()

        #if j%1000 == 0:
            #print(j, loss)
model.eval()
ComputeAccr(test_loader, model)


netname="REAL_1_L{}E{}B{}.pkl".format(int(learning_rate*1000), num_epoch, batch_size)
torch.save(model, netname)
###################### CODE FOR TRAIN  ######################
###################### NOT EXECUTED ######################
    


Files already downloaded and verified
Files already downloaded and verified


<ipython-input-4-fd2dfb5ae496>:85: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  img = Variable(imgs, volatile=True).cuda()


## Accuracy of Test Data : 77.29367065429688


SystemExit: 0